## Tools for NLP - 2

In [1]:
import findspark
findspark.init('/home/venkat/Downloads/spark-3.2.0-bin-hadoop3.2')

from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [4]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [25]:
sentenceData = spark.createDataFrame([
        (0,'Hi I am learning Spark'),
        (1, 'I like spark'),
        (2, 'Linear regression models are good')
], ['label','sentence'])

In [26]:
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|    0|Hi I am learning ...|
|    1|        I like spark|
|    2|Linear regression...|
+-----+--------------------+



In [27]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [28]:
words_data = tokenizer.transform(sentenceData)

In [29]:
words_data.show(truncate=False)

+-----+---------------------------------+---------------------------------------+
|label|sentence                         |words                                  |
+-----+---------------------------------+---------------------------------------+
|0    |Hi I am learning Spark           |[hi, i, am, learning, spark]           |
|1    |I like spark                     |[i, like, spark]                       |
|2    |Linear regression models are good|[linear, regression, models, are, good]|
+-----+---------------------------------+---------------------------------------+



In [30]:
hashing_tf=HashingTF(inputCol='words', outputCol='rawFeatures')

In [31]:
featurized_data = hashing_tf.transform(words_data)

In [32]:
featurized_data.select('rawFeatures').show(truncate = False)

+-----------------------------------------------------------------+
|rawFeatures                                                      |
+-----------------------------------------------------------------+
|(262144,[19036,33808,163984,173558,186845],[1.0,1.0,1.0,1.0,1.0])|
|(262144,[19036,173558,208258],[1.0,1.0,1.0])                     |
|(262144,[22249,46243,58267,113432,160975],[1.0,1.0,1.0,1.0,1.0]) |
+-----------------------------------------------------------------+



In [33]:
idf = IDF(inputCol = 'rawFeatures', outputCol='features')

In [34]:
idf_model = idf.fit(featurized_data)

In [35]:
rescaled_data = idf_model.transform(featurized_data)

In [36]:
rescaled_data.select('label','features').show(truncate=False)

21/11/30 12:49:54 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
21/11/30 12:49:54 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


+-----+----------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                      |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------+
|0    |(262144,[19036,33808,163984,173558,186845],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.28768207245178085,0.6931471805599453])|
|1    |(262144,[19036,173558,208258],[0.28768207245178085,0.28768207245178085,0.6931471805599453])                                                   |
|2    |(262144,[22249,46243,58267,113432,160975],[0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])   |
+-----+---------------------------------------------------------------------------------------

In [24]:
# Collect documents into vectors
from pyspark.ml.feature import CountVectorizer

In [39]:
df = spark.createDataFrame([
        (0,"a b c".split(" ")),
        (1,"a b b c a".split(" "))
],["id", "words"])

In [40]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [42]:
cv = CountVectorizer(inputCol='words', outputCol='features', vocabSize=3, minDF = 2.0)

In [43]:
model = cv.fit(df)

In [44]:
result = model.transform(df)

In [45]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

